# Creating word vectors using word2vec implementation in Spark MLlib

### Here we show how to use Spark MLlib Word2Vec for generating word-features 
#### The data being used is the attack comments text data

Depeding on the size of your DSVM, this notebook may take about 1-2 mins to finish

MLlib Word2Vec: https://spark.apache.org/docs/2.2.0/mllib-feature-extraction.html#word2vec

In [1]:
!cd /home/remoteuser/notebooks/SparkML
!rm -r Outputs
!mkdir Outputs

!cd /home/remoteuser/notebooks/SparkML/Data
!rm -r text_data.zip text_data
!wget https://activelearning.blob.core.windows.net/activelearningdemo/text_data.zip
!unzip text_data.zip

!cd /home/remoteuser/notebooks/SparkML/pySpark

--2018-03-02 17:06:13--  https://activelearning.blob.core.windows.net/activelearningdemo/text_data.zip
Resolving activelearning.blob.core.windows.net (activelearning.blob.core.windows.net)... 13.77.184.64
Connecting to activelearning.blob.core.windows.net (activelearning.blob.core.windows.net)|13.77.184.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64260679 (61M) [application/x-zip-compressed]
Saving to: ‘text_data.zip’

100%[======================================>] 64,260,679  25.8MB/s   in 2.4s   

2018-03-02 17:06:17 (25.8 MB/s) - ‘text_data.zip’ saved [64260679/64260679]

Archive:  text_data.zip
   creating: text_data/
  inflating: text_data/aggression_data.csv  
  inflating: text_data/attack_data.csv  
  inflating: text_data/toxicity_data.csv  


## Set directory path for input data 
#### Input data is downloaded locally to a DSVM

In [2]:
# 1. Location of training data on 
text_file = "/home/remoteuser/notebooks/SparkML/Data/text_data/attack_data.csv"

## Set spark context and import necessary libraries

In [3]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession, DataFrame, SparkSession
from pyspark.sql.functions import UserDefinedFunction, regexp_replace, trim, col, lower, lit, udf, monotonically_increasing_id
from pyspark.sql.types import *
from pyspark.ml.feature import Word2Vec, Word2VecModel, Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.ml.linalg import Vectors

import numpy as np
import datetime
import pandas as pd


sqlContext = SQLContext(sc)

## Data ingestion: Read in attack text csv data

In [4]:
## READ IN DATA AND CREATE SPARK DATAFRAME FROM A CSV
text_df = spark.read.csv(path=text_file, header=True, inferSchema=True, sep=",")
text_df.cache(); text_df.count();

## REGISTER DATA-FRAME AS A TEMP-TABLE IN SQL-CONTEXT
text_df.createOrReplaceTempView("text_table")

text_df.head(1)

[Row(rev_id=37675, comment="`-NEWLINE_TOKENThis is not ``creative``.  Those are the dictionary definitions of the terms ``insurance`` and ``ensurance`` as properly applied to ``destruction``.  If you don't understand that, fine, legitimate criticism, I'll write up ``three man cell`` and ``bounty hunter`` and then it will be easy to understand why ``ensured`` and ``insured`` are different - and why both differ from ``assured``.NEWLINE_TOKENNEWLINE_TOKENThe sentence you quote is absolutely neutral.  You just aren't familiar with the underlying theory of strike-back (e.g. submarines as employed in nuclear warfare) guiding the insurance, nor likely the three man cell structure that kept the IRA from being broken by the British.  If that's my fault, fine, I can fix that to explain.  But ther'es nothing ``personal`` or ``creative`` about it.NEWLINE_TOKENNEWLINE_TOKENI'm tired of arguing with you.  Re: the other article, ``multi-party`` turns up plenty, and there is more use of ``mutually`` t

## Select and filter data set

In [5]:
### SELCT ONLY REV_ID AND COMMENT FIELDS, AND FILTER FOR TRAINING DATA AND ARTILES ONLY
sqlStatement = """ SELECT rev_id, comment 
            FROM text_table 
            where ns = 'article' and split = 'train' """
text_filtered_df = spark.sql(sqlStatement)

## CACHE NEW DATAFRAME IN MEMORY AND CREATE TEMPORARY TABLE
text_filtered_df.cache(); text_filtered_df.count();
text_filtered_df.createOrReplaceTempView("text_filtered_table")

## COUNT NUMBER OF ROWS IN DATAFRAME 
text_filtered_df.count()

31253

## Lowercase and remove some words and remove punctuations

In [6]:
text_filtered_df2 = text_filtered_df.withColumn("comment1", lower(col("comment"))).\
    withColumn("comment2", regexp_replace("comment1", '-newline_token', "")).\
    withColumn("comment3", regexp_replace("comment2", 'newline_token', "")).\
    withColumn("comment_final", regexp_replace("comment3", '[^\w-_ ]', "")).\
    select('rev_id', 'comment_final')
    
text_filtered_df2.head(1)

[Row(rev_id=37675, comment_final='this is not creative  those are the dictionary definitions of the terms insurance and ensurance as properly applied to destruction  if you dont understand that fine legitimate criticism ill write up three man cell and bounty hunter and then it will be easy to understand why ensured and insured are different - and why both differ from assuredthe sentence you quote is absolutely neutral  you just arent familiar with the underlying theory of strike-back eg submarines as employed in nuclear warfare guiding the insurance nor likely the three man cell structure that kept the ira from being broken by the british  if thats my fault fine i can fix that to explain  but theres nothing personal or creative about itim tired of arguing with you  re the other article multi-party turns up plenty and there is more use of mutually than mutual  if i were to apply your standard id be moving mutual assured destruction to talk for not appealing to a reagan voters biases abo

## Tokenize and remove stopwords

In [7]:
## DEFINE TOKENIZER AND STOPWORD REMOVER
tokenizer = Tokenizer(inputCol="comment_final", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtWords")

## TRANSFORM DATASET
text_filtered_df3 = tokenizer.transform(text_filtered_df2)
text_filtered_df4 = remover.transform(text_filtered_df3)
text_filtered_df4.cache(); text_filtered_df4.count();
text_filtered_df4.show(5)

+------+--------------------+--------------------+--------------------+
|rev_id|       comment_final|               words|           filtWords|
+------+--------------------+--------------------+--------------------+
| 37675|this is not creat...|[this, is, not, c...|[creative, , dict...|
| 44816| the term standar...|[, the, term, sta...|[, term, standard...|
| 49851|true or false the...|[true, or, false,...|[true, false, sit...|
| 93890|this page will ne...|[this, page, will...|[page, need, disa...|
|103624|i removed the fol...|[i, removed, the,...|[removed, followi...|
+------+--------------------+--------------------+--------------------+
only showing top 5 rows



## DEFINE AND RUN WORD2VEC ON COMMENTS

In [8]:
model = None
window_size = 5
vector_size = 50
min_count = 5

word2Vec = Word2Vec(windowSize = window_size, vectorSize = vector_size, minCount = min_count, inputCol="filtWords", outputCol="result")
model = word2Vec.fit(text_filtered_df4)

## Examine some words, and other words close to them from these feature neighborhood

In [9]:
model.findSynonyms("bad", 20).select("word").head(5)

[Row(word='assume'),
 Row(word='good'),
 Row(word='wikipediaassume'),
 Row(word='taste'),
 Row(word='luck')]

## Examine how vector features look like

In [10]:
word2vec_features = model.getVectors().select("*")
word2vec_features.head(1)

[Row(word='quotient', vector=DenseVector([0.0158, -0.0356, -0.0354, 0.0144, 0.0033, 0.034, 0.0258, 0.0013, 0.0524, 0.0007, 0.012, -0.0477, -0.0017, -0.0229, 0.0558, 0.0228, -0.0236, 0.0211, 0.046, 0.0169, 0.0605, -0.0035, -0.0193, -0.0111, 0.0192, 0.0359, -0.017, 0.0169, 0.0165, 0.0212, -0.005, -0.0028, -0.0086, -0.0314, -0.0292, -0.0268, 0.0252, -0.0314, -0.0122, -0.0042, 0.0108, -0.0148, -0.0142, -0.0102, -0.0495, 0.0307, -0.0126, 0.0221, 0.0126, 0.0325]))]

## Convert Spark DF to Pandas DF

In [11]:
word2vec_features_pdf = word2vec_features.toPandas()
word2vec_features_pdf.head(4)

,word,vector
0,quotient,"[0.0158357284963, -0.035588581115, -0.03541030..."
1,incident,"[0.065705947578, -0.0430361256003, 0.071241065..."
2,serious,"[0.0902081876993, -0.112241379917, 0.021617949..."
3,wgbh,"[0.0220530647784, 0.00511172134429, 0.00809607..."


## Save features in CSV for subsequent steps

In [12]:
word2vec_features_pdf.to_csv('/home/remoteuser/notebooks/SparkML/Outputs/Word2Vec-Features.csv')

## Getting comment-level vectors from word-level vectors (averaging)

In [13]:
comment_vectors_df = model.transform(text_filtered_df4).select('rev_id','result')
comment_vectors_df.head(2)

[Row(rev_id=37675, result=DenseVector([0.0059, -0.0168, 0.0592, -0.0123, -0.0115, 0.0423, 0.0462, -0.0375, -0.0364, 0.0148, -0.0223, -0.0288, -0.0024, 0.0432, 0.0141, -0.0086, -0.0121, 0.0061, -0.0014, -0.0183, -0.0375, 0.0758, 0.0331, -0.0504, -0.0619, 0.0418, 0.01, 0.0169, -0.0038, 0.0287, 0.0136, 0.0074, 0.0038, 0.0674, -0.0006, 0.0636, 0.0086, -0.0379, -0.0296, 0.0875, 0.0012, -0.0048, 0.0342, -0.0377, 0.0147, 0.0264, 0.0349, 0.0141, 0.037, 0.0008])),
 Row(rev_id=44816, result=DenseVector([0.0011, -0.0079, 0.08, -0.0381, -0.0096, 0.025, 0.0441, -0.0372, -0.0436, 0.005, -0.0485, -0.0313, 0.0078, 0.053, 0.0081, -0.0128, -0.0097, 0.0052, 0.019, -0.0011, -0.035, 0.0991, 0.0442, -0.046, -0.0726, 0.0351, -0.0163, 0.0386, -0.0077, 0.0457, -0.006, -0.0016, 0.0145, 0.037, -0.0049, 0.0624, 0.035, -0.0276, 0.0007, 0.0772, 0.0046, -0.0162, 0.0126, -0.0399, 0.0124, 0.0549, 0.0351, 0.0244, 0.0504, -0.0094]))]